In [ ]:
# IMPORTS
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input,optimizers, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers, Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.callbacks import ModelCheckpoint
from collections import namedtuple
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime, time
import os, sys
import tqdm
import gc
from keras import callbacks
from multiprocessing import Process
Point = namedtuple('Point', ('x', 'y'))
Circle = namedtuple('Circle', ('r'))
Square = namedtuple('Square', ('side'))
Rectangle = namedtuple('Rectangle', ('length', 'width'))
PointWithDistance = namedtuple('PointWithDistance', ('p', 'dist'))
float_memory_used = 'float32'

In [ ]:
# INIT
# PART 1
number_samples = [128, 256, 512, 1024, 4096] 
validation_size, noise_floor = 0.2, -90.0
su_power = 0 # this is not actually su power just a number to show there is an SU in its image
max_x, max_y, number_image_channels, su_szie = 299, 299, 8, 60  # su_size:30 for 1000, 10 for 100
cell_size, pixel_expansion = 1000 / max_x, max_x / 100
pu_shape, su_shape = 'circle', 'circle' # shape = {'circle', 'square', 'point'}
style = "raw_power_min_max_norm"  # {"raw_power_zscore_norm", "image_intensity", "raw_power_min_max_norm"}
intensity_degradation, slope, su_slope = 'log', 5, 5  # 'log', 'linear', slope 3 for 1000, 5 for 100
max_pus_num, max_sus_num = 20, 10
propagation_model = 'splat' # 'splat', 'log', 'testbed'
noise, std = False, 1 # False for splat
if su_shape == 'circle':
    su_param = Circle(su_szie)
elif su_shape == 'square':
    su_param = Square(su_szie)
else:
    su_param = None
    
sensors = False
if sensors:
    sensors_num = 225
    sensors_file_path = f"data/sensors/square{100}/{sensors_num}/sensors.txt"

# PART 2
number_of_proccessors = 12
memory_size_allowed = 4 # in Gigabyte
float_size = 0
if float_memory_used == "float16":
    float_size = 16
elif float_memory_used == "float" or "float32":
    float_size = 32
elif float_memory_used == "float8":
    float_size = 8


batch_size = int(memory_size_allowed / (max_x * max_y * number_image_channels * float_size/(8 * 1024 ** 3)))


dtime = datetime.datetime.now().strftime('_%Y%m_%d%H_%M')
color = "color" if number_image_channels > 1 else "gray"
image_dir = 'ML/data/pictures_' + str(max_x) + '_' + str(max_y) + '_transfer/' + propagation_model + (
    "/noisy_std_" + str(std) if noise else "") + '/pu_' + pu_shape + '_su_' + su_shape + '_' + (
    "" if su_shape == 'point' else str(su_szie)) + "/" + style + "/" + color +'/' + (
    "" if pu_shape == 'point' and su_shape == 'point' else (f"{intensity_degradation}_pu{slope}_su{su_slope}")) + (
    "/" + str(sensors_num) + "sensors" if sensors else f"/{max_pus_num}pus") + \
        f"_{max_sus_num}sus_{number_image_channels}channels" + "/images"

if not os.path.exists(image_dir):
        os.makedirs(image_dir)

In [ ]:
image_dir = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/pictures_299_299_transfer/log/" +\
            "noisy_std_1/pu_circle_su_circle_60/raw_power_min_max_norm/color/log_pu5_su6/" +\
            "variable_sensors_10_20_pus_5_sus_8_channels/images"
sensors_location = {}
for sensor_num in [49, 100, 225, 400, 625]:
    sensors_location[sensor_num] = []
    with open(f"data/sensors/square{100}/{sensor_num}/sensors.txt", 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.split(',')
            sensors_location[sensor_num].append(Point(int(float(line[0])), int(float(line[1]))))

In [ ]:
# LOAD DATA
MAX_SU_TOTAL = False
num_columns = (sensors_num + 1 if sensors else max_pus_num * 3 + 1) + max_sus_num * 3 + 2
# num_columns = max_pus_num * 3 + 1 + max_sus_num * 3 + 2
cols = [i for i in range(num_columns)]
dataset_name = "dynamic_pus_using_pus_70000_min10_max20PUs_1SUs_square100grid_splat_2021_10_16_04_46.txt"
max_dataset_name = "dynamic_pus_max_power_70000_min10_max20PUs_1SUs_square100grid_splat_2021_10_16_04_46.txt"
if MAX_SU_TOTAL:
    max_su_total_dataset_name = "dynamic_pus_maximum_total_sus50000_min10_max20PUs_5SUs_square100grid_splat_2022_06_09_13_24.txt"
with open('/'.join(image_dir.split('/')[:-1]) + '/datasets' + dtime + '.txt', 'w') as set_file:
    set_file.write(dataset_name + "\n")
    set_file.write(max_dataset_name)
    if MAX_SU_TOTAL:
        set_file.write(max_su_total_dataset_name)

dataframe = pd.read_csv('data/' 
                        + dataset_name, delimiter=',', header=None, names=cols)
dataframe_max = pd.read_csv('data/' 
                            + max_dataset_name, delimiter=',', header=None)
if MAX_SU_TOTAL:
    dataframe_max_su_total = pd.read_csv('data/' + max_su_total_dataset_name, delimiter=",", header=None,
                                        names=[i for i in range(max_sus_num * 3 + 1)])

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)
if MAX_SU_TOTAL:
    dataframe_max_su_total.reset_index(drop=True, inplace=True)
dataframe_max[dataframe_max.shape[1] - 1] = dataframe_max[dataframe_max.shape[1] - 1].astype(float)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1:]]], axis=1,
                        ignore_index=True)

idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)
if MAX_SU_TOTAL:
    dataframe_max_su_total.drop(idx, inplace=True)

data_reg = dataframe_tot.values
data_reg[data_reg < noise_floor] = noise_floor
if MAX_SU_TOTAL:
    data_max_su_tot = dataframe_max_su_total.values

if sensors:
    sensors_location = []
    with open(sensors_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.split(',')
            sensors_location.append(Point(int(float(line[0])), int(float(line[1]))))
del dataframe, dataframe_tot, dataframe_max

In [ ]:
def euclidian_distance(p1: Point, p2: Point):
    return ((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5 * cell_size

def calculate_mu_sigma(data, num_pus):
    sum_non_noise = 0
    for pu_n in range(num_pus): # calculate mu
        sum_non_noise += data[pu_n*3+2]
    mu = ((max_x * max_y - num_pus) * noise_floor + sum_non_noise)/(max_x * max_y)
    sum_square = 0
    for pu_n in range(num_pus): # calculate sigma
        sum_square += (data[pu_n*3+2]-mu)**2
    sum_square += (max_x * max_y - num_pus) * (noise_floor - mu)**2
    sigma = math.sqrt(sum_square/(max_x * max_y))
    return mu, sigma

def get_pu_param(pu_shape: str, intensity_degradation: str, pu_p: float, noise_floor: float, slope: float):
    pu_param = None
    if pu_shape == 'circle':
        if intensity_degradation == "linear":
            pu_param = Circle(int((pu_p - noise_floor) / slope)) # linear
        elif intensity_degradation == "log":
            pu_param = Circle(int(10 ** ((pu_p - noise_floor) / (10 *slope)))) # log_based
    elif pu_shape == 'square':
        if intensity_degradation == "linear":
            pu_param = Square(int(2 ** 0.5 * (pu_p - noise_floor) / slope)) # linear
        elif intensity_degradation == "log":
            pu_param = Square(int(2 ** 0.5 * 10 ** ((pu_p - noise_floor) / (10 *slope)))) # log_based
    elif pu_shape == 'point':
        pu_param = None
    else:
        raise ValueError("Unsupported PU shape(create_image)! ", pu_shape)
    return pu_param

def create_image(data, slope, sensors_num, style="raw_power_z_score", noise_floor=-90, pu_shape= 'circle',
                 pu_param=None, su_shape='circle', su_param=None, intensity_degradation="log", 
                 max_pu_power: float=0, max_su_power: float=0):  
    # style = {"raw_power_zscore_norm", "image_intensity", "raw_power_min_max_norm"}
    # intensity_degradation= {"log", "linear"}
    # if param is None, it's automatically calculated. Highest brightness(or power value) (255 or 1.) would
    # assigned to the center(PU location) and radius(side) would be calculated based on its power, slope, and noise floor.
    # If it is given, intensity(power) of pixel beside center would be calculated in the same fashin with an exception that 
    # intensity below zero(noise_floor) would be replaced by zero(noise_floor)
    if style == "raw_power_min_max_norm":
        # In this way, PUs' location are replaced with their power(dBm) and the power would fade with 
        # slope till gets noise_floor(in circle shape)
        
        # creating pu matrix
        image = np.zeros((max_x, max_y, number_image_channels), dtype=float_memory_used)
        if not sensors:
            pus_num = int(data[0])
#             print(pus_num)
            for pu_i in range(pus_num):
                pu_x = max(0, min(max_x-1, int(data[pu_i * 3 + 1] * pixel_expansion))) 
                pu_y = max(0, min(max_x-1, int(data[pu_i * 3 + 2] * pixel_expansion)))
                pu_p = data[pu_i * 3 + 3]
                pu_channel = int(abs(pu_p)//5) if number_image_channels > 3 else 0
#                 print(pu_x, pu_y, pu_p)
                if pu_param is None:
                    pu_param_p = get_pu_param(pu_shape, intensity_degradation, pu_p, noise_floor, slope)
                else:
                    pu_param_p = pu_param
                points = points_inside_shape(center=Point(pu_x, pu_y),
                                             shape=pu_shape, param=pu_param_p)
                for point in points:
                    if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                        if intensity_degradation == "linear":
                            image[int(abs(pu_p))//10][point.p.x][point.p.y] += (pu_p - slope * point.dist - noise_floor)/(
                                max_pu_power - noise_floor)
                        elif intensity_degradation == "log":
                            if point.dist < 1:
                                image[point.p.x][point.p.y][pu_channel] += (pu_p - noise_floor) / (max_pu_power - noise_floor)
                            else:
                                image[point.p.x][point.p.y][pu_channel] += (pu_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                    max_pu_power - noise_floor)
        else:
            ss_channels_num = number_image_channels - 2
            ss_param, ss_shape = pu_param, pu_shape
            sensors_num = int(data[0])
            sensors_num_at_each_row = int(sensors_num ** 0.5)
            for ss_i in range(sensors_num):
                ss_x = max(0, min(max_x-1, int(sensors_location[sensors_num][ss_i].x * pixel_expansion)))
                ss_y =  max(0, min(max_x-1, int(sensors_location[sensors_num][ss_i].y * pixel_expansion)))
                ss_p = max(noise_floor, data[ss_i+1])
                ss_row, ss_cols = ss_i // sensors_num_at_each_row, ss_i % sensors_num_at_each_row
                ss_channel = ss_cols % ss_channels_num
                if ss_row % 2:
                    ss_channel = (ss_channel + ss_channels_num // 2) % ss_channels_num
                if ss_param is None:
                    ss_param_p = get_pu_param(ss_shape, intensity_degradation, ss_p, noise_floor, slope)
                else:
                    ss_param_p = ss_param
                points = points_inside_shape(center=Point(ss_x, ss_y), shape=ss_shape, param=ss_param_p)
                for point in points:
                    if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                        if intensity_degradation == "linear":
                            image[point.p.x][point.p.y][ss_channel] += (ss_p - slope * point.dist - noise_floor)/(
                                max_pu_power - noise_floor)
                        elif intensity_degradation == "log":
                            if point.dist < 1:
                                image[point.p.x][point.p.y][ss_channel] += (ss_p - noise_floor) / (max_pu_power - noise_floor)
                            else:
                                image[point.p.x][point.p.y][ss_channel] += (ss_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                    max_pu_power - noise_floor)
        del points
        # creating su matrix
        su_num_idx = sensors_num + 1 if sensors else (pus_num * 3 + 1)
        su_num = int(data[su_num_idx])
#         print(su_num)
#         su_num = (len(data) - pus_num * (3 if not sensors else 1)) // 2
#         if not (len(data) - pus_num * (3 if not sensors else 1)) % 2:
#             raise ValueError("Data provided is not correct; can't get SUs' information(create_image)")
        if su_param is None:
            # if su_param is unavailable, a circle(square) with radius(side) 1 is created
            if su_shape == 'circle':
                su_param = Circle(1)
            elif su_shape == 'square':
                su_param = Square(1)
            elif su_shape == 'point':
                su_param = None
            else:
                raise ValueError("Unsupported SU shape(create_image)! ", su_shape)
        
        for su_i in range(su_num - 1):
            su_x = max(0, min(max_x-1, int(data[su_num_idx + su_i * 3 + 1] * pixel_expansion)))
            su_y = max(0, min(max_x-1, int(data[su_num_idx + su_i * 3 + 2] * pixel_expansion)))
            su_p = data[su_num_idx + su_i * 3 + 3]
#             su_p = su_intensity
            su_param_p = get_pu_param(su_shape, intensity_degradation, su_p, noise_floor, su_slope)
            points = points_inside_shape(center=Point(su_x, su_y),
                                         param=su_param_p, shape=su_shape)
            su_channel = 0 if number_image_channels == 1 else -2
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                            su_val = (su_p - su_slope * point.dist - noise_floor)/(max_su_power - noise_floor)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            su_val = (su_p - noise_floor) / (max_su_power - noise_floor)
                        else:
                            su_val = (su_p - su_slope * 10*math.log10(point.dist) - noise_floor)/(
                                max_su_power - noise_floor)
                    image[point.p.x][point.p.y][su_channel] += su_val
            del points
        # the last and  target SU
        su_intensity = 1.
        su_x = max(0, min(max_x-1, int(data[su_num_idx + (su_num - 1) * 3 + 1] * pixel_expansion)))
        su_y = max(0, min(max_x-1, int(data[su_num_idx + (su_num - 1) * 3 + 2] * pixel_expansion)))
        points = points_inside_shape(center=Point(su_x, su_y),
                                     param=su_param, shape=su_shape)
        su_channel = -1
        for point in points:
            if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                image[point.p.x][point.p.y][su_channel] += su_intensity
        del points
        return image
        
    elif style == "image_intensity":
        # creating PU image
        image = np.zeros((1,number_image_channels,max_x, max_y), dtype=float_memory_used)
        pus_num = int(data[0])
        for pu_i in range(pus_num):
            pu_x = max(0, min(max_x-1, int(data[pu_i * 3 + 1] * pixel_expansion))) 
            pu_y = max(0, min(max_x-1, int(data[pu_i * 3 + 2] * pixel_expansion)))
            pu_p = data[pu_i * 3 + 3]
            
            if pu_param is None:
                pu_param_p = get_pu_param(pu_shape, intensity_degradation, pu_p, noise_floor, slope)
            else:
                pu_param_p = pu_param
            
            points = points_inside_shape(center=Point(pu_x, pu_y), shape=pu_shape, param=pu_param_p)
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                        image[0][0][point.p.x][point.p.y] += (pu_p - slope * point.dist - noise_floor)/(
                            max_pu_power - noise_floor)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            image[0][0][point.p.x][point.p.y] += (pu_p - noise_floor) / (max_pu_power - noise_floor)
                        else:
                            image[0][0][point.p.x][point.p.y] += (pu_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                max_pu_power - noise_floor)
                        
        # creating SU image
        del points
        # creating su matrix
        su_num_idx = sensors_num if sensors else (pus_num * 3 + 1)
        su_num = int(data[su_num_idx])
        if su_param is None:
            # if su_param is unavailable, a circle(square) with radius(side) 1 is created
            if su_shape == 'circle':
                su_param = Circle(1)
            elif su_shape == 'square':
                su_param = Square(1)
            elif su_shape == 'point':
                su_param = None
            else:
                raise ValueError("Unsupported SU shape(create_image)! ", su_shape)
        
        for su_i in range(su_num - 1):
            su_x = max(0, min(max_x-1, int(data[su_num_idx + su_i * 3 + 1])))
            su_y = max(0, min(max_x-1, int(data[su_num_idx + su_i * 3 + 2])))
            su_p = data[su_num_idx + su_i * 3 + 3]
            
#             su_p = su_intensity
            points = points_inside_shape(center=Point(su_x, su_y), param=su_param, shape=su_shape)
            su_channel = 0 if number_image_channels == 1 else -1
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                            su_val = (su_p - slope * point.dist - noise_floor)/(max_pu_power - noise_floor)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            su_val = (su_p - noise_floor) / (max_pu_power - noise_floor)
                        else:
                            su_val = (su_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                max_pu_power - noise_floor)
                    image[0][su_channel][point.p.x][point.p.y] += su_val
            del points
        # the last and  target SU
        su_intensity = 1.
        su_x = max(0, min(max_x-1, int(data[su_num_idx + (su_num - 1) * 3 + 1])))
        su_y = max(0, min(max_x-1, int(data[su_num_idx + (su_num - 1) * 3 + 2])))
#         print(su_x, su_y)
        points = points_inside_shape(center=Point(su_x, su_y), param=su_param, shape=su_shape)
        su_channel = 0 if number_image_channels == 1 else -1
        for point in points:
            if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                image[0][su_channel][point.p.x][point.p.y] += su_intensity
        del points
        return image       
            
    else:
        raise ValueError("Unsupported style(create_image)! ", style)
        
def points_inside_shape(center: Point, shape: str, param)-> list:
    # This function returns points+distance around center with defined shape
    if shape == 'circle':
        # First creates points inside a square(around orgigin) with 2*r side and then remove those with distance > r.
        # Shift all remaining around center. O(4r^2)
        r, origin = param.r, Point(0, 0)
        square_points = set((Point(x, y) for x in range(max(-int(r/cell_size), -max_x), 
                             min(int(r/cell_size), max_x) + 1) 
                             for y in range(max(-int(r/cell_size), -max_y), min(int(r/cell_size), max_y) + 1)))
        points = []
        while square_points:
            p = square_points.pop()
            dist = euclidian_distance(p, origin)
            if dist <= r:
                points.append(PointWithDistance(Point(p.x + center.x, p.y + center.y), dist))
                if p.x != 0:
                    points.append(PointWithDistance(Point(-p.x + center.x, p.y + center.y), dist))
                    square_points.remove(Point(-p.x, p.y))
                if p.y != 0:
                    points.append(PointWithDistance(Point(p.x + center.x, -p.y + center.y), dist))
                    square_points.remove(Point(p.x, -p.y))
                if p.x != 0 and p.y != 0:
                    points.append(PointWithDistance(Point(-p.x + center.x, -p.y + center.y), dist))
                    square_points.remove(Point(-p.x, -p.y))
        del square_points
        return points
    elif shape == 'square':
        half_side = param.side // 2
        return [PointWithDistance(Point(x, y), euclidian_distance(Point(x, y), center)) for x in range(-half_side + center.x,
                                                                                               half_side + center.x+1) 
                         for y in range(-half_side + center.y, half_side + center.y + 1)]
    elif shape == 'point':
        return [PointWithDistance(center, 0)]
    else:
        raise ValueError("Unsupported shape(points_inside_shape)! ", shape)
        
def read_image(image_num, image_dir=image_dir):
    if False and style == "image_intensity":
        image = plt.imread(image_dir + '/image' + str(image_num)+'.png')
        image = np.swapaxes(image, 0, 2)
        image = np.array(image[:number_image_channels], dtype=float_memory_used).reshape(1, number_image_channels, max_x, max_y)
    elif  style == "raw_power_min_max_norm" or style == "raw_power_zscore_norm" or style == "image_intensity":
        suffix = 'npz'  # npy, npz
#         image = np.load(f"{image_dir}/images{image_num//100000}/image{image_num}.{suffix}") 
        image = np.load(f"{image_dir}/image{image_num}.{suffix}") 
        if type(image) == np.lib.npyio.NpzFile:
            image = image['a']
    
    return image

In [ ]:
# Creating synthesized images
def creating_image(start, end):
    for image_num in tqdm.tqdm(range(start, end+1)):
        image = create_image(data=data_reg_train[image_num], slope=slope, style=style, 
                             noise_floor=noise_floor,
                             pu_shape=pu_shape, su_shape=su_shape, su_param=su_param, 
                             sensors_num=(sensors_num if sensors else 0), 
                             intensity_degradation=intensity_degradation, 
                             max_pu_power=0.0 if not sensors else -60)
        if new_image_state[image_num] == "rot":
            image = np.rot90(image, 2)
        elif new_image_state[image_num] == "lr":
            image = np.fliplr(image)
        elif new_image_state[image_num] == "ud":
            image = np.flipud(image)
        np.savez_compressed(image_dir + '/aug/image' + str(image_num), a=np.expand_dims(image,0 ))
        
        del image
        
train_size = 2048
data_reg_train = np.repeat(data_reg[:train_size], 4, axis=0)
image_state = ["", "rot", "lr", "ud"] * train_size
p = np.random.permutation(train_size*4)
data_reg_train = data_reg_train[p]
new_image_state = []
for idx in range(train_size*4):
    new_image_state.append(image_state[p[idx]])

In [ ]:
jobs = []
proc_sizes = [data_reg_train.shape[0]//number_of_proccessors] * (number_of_proccessors)
proc_sizes[-1] += data_reg_train.shape[0]%number_of_proccessors
proc_idx = [(sum(proc_sizes[:i]), sum(proc_sizes[:i+1])-1) for i in range(number_of_proccessors)]

for i in range(number_of_proccessors):
    p = Process(target=creating_image, args=(proc_idx[i][0], proc_idx[i][1]))
    jobs.append(p)
    p.start()
for i in range(number_of_proccessors):
    jobs[i].join()

for i in range(number_of_proccessors):
    jobs[i].terminate()
    jobs[i].close()
del jobs

In [ ]:
# Saving images once to save time
# run this cell just for creating images
def creating_image(start, end):
    for image_num in tqdm.tqdm(range(start, end+1)):
        image = create_image(data=data_reg[image_num], slope=slope, style=style, 
                             noise_floor=noise_floor,
                             pu_shape=pu_shape, su_shape=su_shape, su_param=su_param, 
                             sensors_num=(sensors_num if sensors else 0), 
                             intensity_degradation=intensity_degradation, 
                             max_pu_power=0.0 if not sensors else -60,
                             max_su_power=40.0)
        if False and style == "image_intensity":
            if number_image_channels != 3:
                image = np.append(np.array(image[0]), np.zeros((3-number_image_channels,max_x, max_y), 
                                                               dtype=float_memory_used), axis=0)
            image_save = np.swapaxes(image, 0, 2)
            plt.imsave(image_dir + '/image' + str(image_num)+'.png', image_save)
        elif style == "raw_power_min_max_norm" or style == "raw_power_zscore_norm" or style == "image_intensity":
            np.savez_compressed(f"{image_dir}/images{image_num//100000}/image{image_num}",
                                a=np.expand_dims(image,0))
        del image

In [ ]:
jobs = []
proc_sizes = [data_reg.shape[0]//number_of_proccessors] * (number_of_proccessors)
proc_sizes[-1] += data_reg.shape[0]%number_of_proccessors
proc_idx = [(sum(proc_sizes[:i]), sum(proc_sizes[:i+1])-1) for i in range(number_of_proccessors)]

for i in range(number_of_proccessors):
    p = Process(target=creating_image, args=(proc_idx[i][0], proc_idx[i][1]))
    jobs.append(p)
    p.start()
for i in range(number_of_proccessors):
    jobs[i].join()

for i in range(number_of_proccessors):
    jobs[i].terminate()
    jobs[i].close()
del jobs

In [ ]:
# Xception model
model_name = "ResNet50V2"
base_model = applications.ResNet50V2(include_top=False, weights=None,
                                      input_shape=(max_x, max_y, number_image_channels))
base_model.trainable = True
base_model.summary()

In [ ]:
# log-vgg pretrained - pu-setting
model_name = "log_vgg16"
fp_penalty_coef, fn_penalty_coef = 1, 1
model_path = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/pictures_299_299_transfer/log/noisy_std_1/" + \
             "pu_circle_su_circle_60/raw_power_min_max_norm/color/log_pu5_su6/20pus_5sus_8channels/models/vgg16/" + \
             "700000/best_model_lambda_0.1.h5"
model_path = "ML/data/pictures_299_299_transfer/log/noisy_std_1/pu_circle_su_circle_60/" + \
             "raw_power_min_max_norm/color/log_5/pus_1_sus_3_channels_700k/models/700000/" + \
             "best_model_lambda_0_fit.h5"
base_model = models.load_model(model_path, 
                              custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                               'fp_mae': fp_mae,
                                               'mae':'mae', 'mse':'mse'})
base_model.trainable = False
base_model = base_model.layers[1]
base_model.summary()

In [ ]:
# log-vgg pretrained - ss-setting
model_name = "ResNet50V2"
fp_penalty_coef, fn_penalty_coef = 1, 1
model_path = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/" +\
             "pictures_299_299_transfer/log/noisy_std_1/pu_circle_su_circle_60/" + \
             "raw_power_min_max_norm/color/log_pu5_su6/" + \
             "variable_sensors_10_20_pus_5_sus_8_channels/models/ResNet152V2/" + \
             "700000/best_model_lambda_0_1.h5"
model_path = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/pictures_299_299_transfer/" +\
             "log/noisy_std_1/pu_circle_su_circle_60/raw_power_min_max_norm/color/log_pu5_su6/"+\
             "20pus_5sus_8channels/models/ResNet50V2/700000/best_model_lambda_0_lr0.01_mae10.725.h5"
base_model = models.load_model(model_path, 
                              custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                               'fp_mae': fp_mae,
                                               'mae':'mae', 'mse':'mse'})
base_model = base_model.layers[1]
base_model.trainable = False
base_model.summary()

In [ ]:
DEEP = True
def cnn_model(kernel_lam, bias_lam):
    inputs = Input(shape=(max_x, max_y, number_image_channels))
    convolution_filter, dense_filter = 'relu', 'linear'
    convolution_init, dense_init = "lecun_normal", "RandomNormal"
    
    
    if True:
        # We make sure that the base_model is running in inference mode here,
        # by passing `training=False`. This is important for fine-tuning, as you will
        # learn in a few paragraphs.
        cnn = base_model(inputs, training=False)
        cnn = layers.GlobalAveragePooling2D()(cnn)
        cnn = layers.Dense(2048, activation=convolution_filter, kernel_regularizer=regularizers.l2(kernel_lam),
                             bias_regularizer=regularizers.l2(bias_lam), kernel_initializer=convolution_init)(cnn)
        cnn = layers.Dropout(0.5)(cnn)
        outputs = layers.Dense(1, activation=dense_filter, kernel_regularizer=regularizers.l2(kernel_lam),
                             bias_regularizer=regularizers.l2(bias_lam), kernel_initializer=dense_init)(cnn)
        return Model(inputs, outputs)
    else:
        data_format="channels_last"
        filter_shape, pool_size = (1, 1), (2,2)
        cnn = layers.Conv2D(512, filter_shape, padding='same', 
                            activation=convolution_filter, data_format=data_format, 
                            kernel_regularizer=regularizers.l2(kernel_lam), 
                            bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.MaxPooling2D(pool_size=pool_size, data_format=data_format)(cnn)
        
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter,
                            data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.MaxPooling2D(pool_size=pool_size, data_format=data_format)(cnn)
        
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter,
                            data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.MaxPooling2D(pool_size=pool_size, data_format=data_format)(cnn)
        cnn = layers.Conv2D(512, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(128, filter_shape,padding='same', activation=convolution_filter,
                            data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(32, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(8, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.Conv2D(1, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer=convolution_init)(cnn)
        cnn = layers.GlobalAveragePooling2D()(cnn)
        return Model(inputs, cnn)
    

class DataBatchGenerator(Sequence):
    def __init__(self, dataset:np.ndarray, batch_size:int, start_idx:int,
                 number_image_channels:int,
                 max_x, max_y, float_memory_used, image_dir = image_dir, conserve=0):
        self.dataset, self.batch_size, self.start_idx = dataset, batch_size, start_idx
        self.number_image_channels, self.max_x, self.max_y = number_image_channels, max_x, max_y
        self.float_memory_used = float_memory_used
        self.conserve = conserve
        self._image_dir = image_dir
    
    def __len__(self):
        return np.ceil(self.dataset.shape[0] / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        size = min(self.dataset.shape[0] - idx * self.batch_size, self.batch_size)
        batch_x = np.empty((size, self.max_x, self.max_y, self.number_image_channels), 
                           dtype=self.float_memory_used)
        batch_y = np.empty((size), dtype=self.float_memory_used)
        for i in range(size):
            batch_x[i] = read_image(self.start_idx + idx * self.batch_size + i, self._image_dir)
            batch_y[i] = self.dataset[idx * self.batch_size + i][- 1 - self.conserve]
        return batch_x, batch_y

class PredictBatchGenerator(Sequence):
    def __init__(self, dataset_size: int, batch_size: int, start_idx: int,
                 number_image_channels: int, max_x: int,
                 max_y: int, float_memory_used,
                 image_dir: str):
        self.dataset_size = dataset_size
        self.batch_size, self.start_idx = batch_size, start_idx
        self.number_image_channels, self.max_x, self.max_y = number_image_channels, max_x, max_y
        self.float_memory_used = float_memory_used
        self.image_dir = image_dir
    
    def __len__(self):
        return np.ceil(self.dataset_size / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        size = min(self.dataset_size - idx * self.batch_size, self.batch_size)
        batch_x = np.empty((size, self.max_x, self.max_y, self.number_image_channels),
                           dtype=self.float_memory_used)
        for i in range(size):
            batch_x[i] = read_image(self.start_idx + idx * self.batch_size + i, self.image_dir)
        return batch_x
        
    
def custom_loss(fp_penalty_coef, fn_penalty_coef):
    # custom loss function that penalize false positive and negative differently
    def loss(y_true, y_pred):
        res = y_pred - y_true
        res = tf.where(res > 0, res * fp_penalty_coef, res * fn_penalty_coef)
        return K.mean(K.square(res))
    return loss

def fp_mae(y_true, y_pred):
    # custom metric that replace false negative with zero and return the mean of new vector
    res = y_pred - y_true
    res = tf.nn.relu(res)
    return K.mean(res)


In [ ]:
# tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

In [ ]:
# CNN: support batching
AUGMENTED = True
TEST, CONSERVE, FINE_TUNING = True, False, True
mini_batch = 32 if max(max_x, max_y) == 1000 else 64
epochs = 35 if max(max_x, max_y) == 1000 else 130
MAX_QUEUE_SIZE, WORKERS = 28, 4
fp_penalty_coef, fn_penalty_coef = 1, 1
hyper_metric, mode = "val_mae", 'min'  # the metric that hyper parameters are tuned with
prev_sample = 0
lambda_vec = [0, 0.01, 0.1, 1] 
# average_diff_power, fp_mean_power = [],[] #[7.177, 8.088, 8.183], [3.438, 3.506, 2.662]
# best_lambda = []
average_diff_power_conserve, fp_mean_power_conserve = [], []
all_cnns = []
if CONSERVE: # for conservative
    prev_number_samples = [0] + number_samples[:-1]

for num_sample_idx, number_sample in enumerate(number_samples):
    if CONSERVE:
        data_reg[prev_number_samples[num_sample_idx]:number_sample, -1] = data_reg[
            prev_number_samples[num_sample_idx]:number_sample, -1] - 1.5 # conserv value
    MODEL_PATH = '/'.join(image_dir.split('/')[:-1]) + '/models/' + ("aug/" if AUGMENTED else "") \
    + model_name + "/" + ("conservative/" if CONSERVE else "") + str(number_sample)
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)
    MODEL_PATH += "/best_model_lambda_"
    if True:
        cnns = [cnn_model(lamb, 0) for lamb in lambda_vec]
        for cnn in cnns:
            cnn.compile(loss=custom_loss(fp_penalty_coef, fn_penalty_coef), 
                        optimizer=optimizers.Adam(), 
                        metrics=['mse', 'mae', fp_mae])
        checkpointers = [ModelCheckpoint(filepath=MODEL_PATH + str(lamb)+ '.h5',
                                         verbose=2, save_best_only=True, 
                                         monitor=hyper_metric,
                                         mode=mode)
                         for lamb in lambda_vec]
    else:
        cnns = []
        cnns = [models.load_model(MODEL_PATH + str(lamb) + '.h5', 
                                  custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                                  'fp_mae': fp_mae }) 
                for lamb in lambda_vec]
    number_start = time.time()
    train_generator = DataBatchGenerator(
        dataset=data_reg_train[:number_sample * 4] if AUGMENTED else data_reg[:number_sample],
        batch_size=mini_batch,
        start_idx=prev_sample,
        number_image_channels=number_image_channels,
        max_x=max_x, max_y=max_y, float_memory_used=float_memory_used,
        image_dir=image_dir + ("/aug" if AUGMENTED else ""))
    

    val_size = math.ceil(number_sample * validation_size)
#     val_size = data_reg.shape[0] - number_sample
    val_generator = DataBatchGenerator(dataset=data_reg[number_sample:number_sample+val_size], 
                                       batch_size=mini_batch,
                                       start_idx=number_sample,
                                       number_image_channels=number_image_channels,
                                       max_x=max_x, max_y=max_y, 
                                       float_memory_used=float_memory_used,
                                      image_dir="/".join(image_dir.split("/")[:-1]) + "/images")
  
    print('number_samples:', number_sample, ", New samples:", number_sample - prev_sample)
    print("Validation size:", val_size, ", starts:", number_sample, ", ends:", number_sample + val_size - 1)
    
    for lamb_idx, lamb in enumerate(lambda_vec):
        lambda_start = time.time()
        cnns[lamb_idx].fit(train_generator, epochs=epochs, verbose=2,
                           validation_data=val_generator, 
                           shuffle=True, callbacks=[checkpointers[lamb_idx], 
                                                    callbacks.EarlyStopping(monitor=hyper_metric, min_delta=1e-2,
                                                                           patience=10,
                                                                           mode=mode)],
#                                                    callbacks.ReduceLROnPlateau(monitor=hyper_metric,
#                                                                               factor=0.2,
#                                                                               patience=10,
#                                                                               mode=mode)], 
                           workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, 
                           use_multiprocessing=False)
        
        print("\nLambda:", lamb, ", Time:", str(datetime.timedelta(seconds=int(time.time() - lambda_start))))
        print("Train Error(all epochs):", min(cnns[lamb_idx].history.history['mae']), '\n', 
              [round(val, 3) for val in cnns[lamb_idx].history.history['mae']])
        print("Train FP Error(all epochs):", min(cnns[lamb_idx].history.history['fp_mae']), '\n',
              [round(val,3) for val in cnns[lamb_idx].history.history['fp_mae']])
        print("Val Error(all epochs):", min(cnns[lamb_idx].history.history['val_mae']), '\n', 
              [round(val,3) for val in cnns[lamb_idx].history.history['val_mae']])
        print("Val FP Error(all epochs):", min(cnns[lamb_idx].history.history['val_fp_mae']), '\n',
              [round(val,3) for val in cnns[lamb_idx].history.history['val_fp_mae']])
    if FINE_TUNING:
    # ******************** fine-tunning *******
        print("******FINE TUNNING ******")
        # reloading the best
        cnns = [models.load_model(MODEL_PATH + str(lamb) + '.h5', 
                                  custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                                   'fp_mae': fp_mae,
                                                   'mae':'mae', 'mse':'mse'}) for lamb in lambda_vec]
        for cnn in cnns:
            cnn.trainable = True
            cnn.compile(loss=custom_loss(fp_penalty_coef, fn_penalty_coef), 
                        optimizer=optimizers.Adam(1e-5), 
                        metrics=['mse', 'mae', fp_mae])
        train_generator = DataBatchGenerator(
            dataset=data_reg_train[:number_sample * 4] if AUGMENTED else data_reg[:number_sample],
            batch_size=mini_batch//2,
            start_idx=prev_sample,
            number_image_channels=number_image_channels,
            max_x=max_x, max_y=max_y, float_memory_used=float_memory_used,
            image_dir=image_dir + ("/aug" if AUGMENTED else ""))
        val_generator = DataBatchGenerator(dataset=data_reg[number_sample:number_sample+val_size], 
                                           batch_size=mini_batch,
                                           start_idx=number_sample,
                                           number_image_channels=number_image_channels,
                                           max_x=max_x, max_y=max_y, 
                                           float_memory_used=float_memory_used,
                                          image_dir="/".join(image_dir.split("/")[:-1]) + "/images")
        for lamb_idx, lamb in enumerate(lambda_vec):
    #     for lamb_idx, lamb in enumerate(lambda_vec[:len(lambda_vec) - num_sample_idx//2]):
    #         if num_sample_idx == 3 and lamb_idx < 4:
    #             continue
            lambda_start = time.time()
            cnns[lamb_idx].fit(train_generator, epochs=int(epochs//2), verbose=2,
                               validation_data=val_generator, 
                               shuffle=True, callbacks=[checkpointers[lamb_idx],
                                                        callbacks.EarlyStopping(
                                                            monitor=hyper_metric, min_delta=1e-2,
                                                            patience=7, mode=mode)], 
                               workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, 
                               use_multiprocessing=False)

            print("\nLambda:", lamb, ", Time:", str(datetime.timedelta(seconds=int(time.time() - lambda_start))))
            print("Train Error(all epochs):", min(cnns[lamb_idx].history.history['mae']), '\n', 
                  [round(val, 3) for val in cnns[lamb_idx].history.history['mae']])
            print("Train FP Error(all epochs):", min(cnns[lamb_idx].history.history['fp_mae']), '\n',
                  [round(val,3) for val in cnns[lamb_idx].history.history['fp_mae']])
            print("Val Error(all epochs):", min(cnns[lamb_idx].history.history['val_mae']), '\n', 
                  [round(val,3) for val in cnns[lamb_idx].history.history['val_mae']])
            print("Val FP Error(all epochs):", min(cnns[lamb_idx].history.history['val_fp_mae']), '\n',
                  [round(val,3) for val in cnns[lamb_idx].history.history['val_fp_mae']])
    
    models_min_mae = [min(cnns[lam_idx].history.history[hyper_metric]) for
                      lam_idx,_ in enumerate(lambda_vec)]
    best_lamb_idx = models_min_mae.index(min(models_min_mae))
    best_lambda.append(lambda_vec[best_lamb_idx])
    print("\nTrainig set size:", number_sample, ", Time:", str(datetime.timedelta(seconds=int(time.time() - 
                                                                                              number_start))),
          ", best_lambda:", lambda_vec[best_lamb_idx], ", min_" , ("fp_" if hyper_metric == "val_fp_mae" else ""),
          "error:", round(min(models_min_mae), 3))
    all_cnns.append(cnns)
    del cnns, train_generator, val_generator
    
    if TEST:
        # evaluating test images
        best_model = None
        best_model = models.load_model(MODEL_PATH + str(lambda_vec[best_lamb_idx]) + '.h5', 
                                       custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                                       'fp_mae': fp_mae,
                                                      'mae':'mae', 'mse':'mse'})
        test_generator = DataBatchGenerator(dataset=data_reg[number_sample + val_size:], 
                                            batch_size=mini_batch * 2,
                                            start_idx=number_sample + val_size, 
                                            number_image_channels=number_image_channels,
                                            max_x=max_x, max_y=max_y, float_memory_used=float_memory_used)

        print("Test starts: ", number_sample + val_size, ", ends: ", data_reg.shape[0] - 1)
        time.sleep(1)
        test_res = best_model.evaluate(test_generator, verbose=1, 
                                       workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, use_multiprocessing=False)
        
        test_mae_idx, test_fp_mae_idx = [best_model.metrics_names.index(mtrc) 
                                         for mtrc in ['mae','fp_mae']]
        test_mae, test_fp_mae = test_res[test_mae_idx], test_res[test_fp_mae_idx]
        average_diff_power.append(round(test_mae, 3))
        fp_mean_power.append(round(test_fp_mae, 3))
        print('average_error: ', average_diff_power[-1], ', fp_average_error: ', 
              fp_mean_power[-1])
        
        if False:
            test_generator_conserve = DataBatchGenerator(dataset=data_reg[number_sample + val_size:], 
                                                         batch_size=mini_batch,
                                                         start_idx=number_sample + val_size, 
                                                         number_image_channels=number_image_channels,
                                                         max_x=max_x, max_y=max_y, 
                                                         float_memory_used=float_memory_used, 
                                                         conserve=1)
            test_res_conserve = best_model.evaluate(test_generator_conserve, verbose=1, 
                                                    workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, 
                                                    use_multiprocessing=False)
            test_mae_cons, test_fp_mae_cons = test_res_conserve[test_mae_idx], test_res_conserve[test_fp_mae_idx]
            average_diff_power_conserve.append(round(test_mae_cons, 3))
            fp_mean_power_conserve.append(round(test_fp_mae_cons, 3))
            print('Conserve, average_error: ', average_diff_power_conserve[-1], ', fp_average_error: ',
                 fp_mean_power_conserve[-1])
        print("\n\n")

        
        var_f = open('/'.join(image_dir.split('/')[:-1]) +  '/models/' + model_name + "/"
                     + intensity_degradation + '_' + str(slope) + '_' + 
                     dtime + ".dat", "wb") # file for saving results
        pickle.dump([average_diff_power, fp_mean_power, number_samples, best_lambda, 
                     dataset_name, max_dataset_name, average_diff_power_conserve, fp_mean_power_conserve,
                     checkpointers],
                    file=var_f)
        var_f.close()
        del best_model, test_generator

In [ ]:
print("average_diff_power:", [round(x, 3) for x in average_diff_power])
print("fp_error:", [round(x, 3) for x in fp_mean_power])
print("best_lambda:", best_lambda)

In [ ]:
# Saving images once to save time
# run this cell just for creating images
def creating_image_max_su_tot(start, end):
    # for image_num in range(115, data_reg.shape[0]):
    # for image_num in range(1625, 5000):
    for row_idx in tqdm.tqdm(range(start, end+1)):  #4463, data_reg.shape[0]
        row_sample = data_reg[row_idx]
        row_sample[int(row_sample[0]) * 3 + 1] = 1
        for su_idx in range(int(data_max_su_tot[row_idx][0])):
            row_sample[int(row_sample[0]) * 3 + 2 : 
                       int(row_sample[0]) * 3 + 4] = data_max_su_tot[row_idx][1 + su_idx * 3: 
                                                                              1 + su_idx * 3 + 2]
            image = create_image(data=row_sample, 
                                 slope=slope, style=style, noise_floor=noise_floor,
                                 pu_shape=pu_shape, su_shape=su_shape, su_param=su_param, 
                                 sensors_num=(sensors_num if sensors else 0), 
                                 intensity_degradation=intensity_degradation, 
                                 max_pu_power=0.0 if not sensors else -60,
                                 max_su_power=40.0)
            if False and style == "image_intensity":
                if number_image_channels != 3:
                    image = np.append(np.array(image[0]), np.zeros((3-number_image_channels,max_x, max_y), 
                                                                   dtype=float_memory_used), axis=0)
                image_save = np.swapaxes(image, 0, 2)
                plt.imsave(max_su_image_dir + '/image' + str(image_num)+'.png', image_save)
            elif style == "raw_power_min_max_norm" or style == "raw_power_zscore_norm" or style == "image_intensity":
        #         np.save(max_su_image_dir + '/image' + str(image_num), image)
    #             np.savez_compressed(f"{image_dir}{(600000 + image_num)//100000}/image{600000 + image_num}",
    #                                 a=np.expand_dims(image,0))
                np.savez_compressed(f"{max_su_image_dir}/image{row_idx * 5 + su_idx}",
                                    a=np.expand_dims(image,0))
            del image

In [ ]:
# Multi-SU using single deep-alloc and another NN
# Create dataset

number_samples = [128, 256, 512, 1024, 2048, 4096]
MAX_QUEUE_SIZE, WORKERS = 28, 4
mini_batch = 128
fp_penalty_coef, fn_penalty_coef = 1, 1
dataset_path = '/'.join(image_dir.split('/')[:-1]) + "/" + model_name

model_path = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/pictures_299_299_transfer/" + \
             "splat/pu_circle_su_circle_60/raw_power_min_max_norm/color/log_pu5_su5/" + \
             "pus_1_sus_3_channels/models/log_vgg16/"
y = np.copy(data_max_su_tot[:,3::3])

for number_sample in number_samples:
    model = models.load_model(f"{model_path}/{number_sample}/best_model_lambda_0.h5",
                              custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef),
                                              'fp_mae': fp_mae,
                                              'mae':'mae', 'mse':'mse'})
    model.trainable = False
    predic_batch_generator = PredictBatchGenerator(dataset_size=data_reg.shape[0] * max_sus_num,
                                                   batch_size=mini_batch,
                                                   image_dir=max_su_image_dir,
                                                   max_x=max_x, max_y=max_y, 
                                                   number_image_channels=number_image_channels,
                                                   start_idx=0, float_memory_used=float_memory_used)
    number_dataset_path = f"{dataset_path}/{number_sample}"
    if not os.path.exists(number_dataset_path):
        os.makedirs(number_dataset_path)
    predict_power = model.predict(predic_batch_generator, verbose=1, 
                                  workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE,
                                  use_multiprocessing=False)
    X = np.copy(data_max_su_tot[:, 1:])
    X[:,2::3] = predict_power.reshape(data_reg.shape[0], max_sus_num)
    
    np.savetxt(f"{number_dataset_path}/X.txt", X, delimiter=",")
    np.savetxt(f"{number_dataset_path}/y.txt", y, delimiter=",")
    print(f"{number_sample} finished.")

In [ ]:
def nn_model(n_inputs: int, n_outputs:int, kernel_lam, bias_lam, num_hidden_layers = 2, num_neurons = 100):
    hidden_filter, last_layer_filter = 'relu', 'linear'
    hidden_init, last_layer_init = "lecun_normal", "RandomNormal"  #he_uniform
    model = Sequential()
    for _ in range(num_hidden_layers):
        model.add(layers.Dense(num_neurons,
                              input_dim=n_inputs, 
                               kernel_initializer=hidden_init,
                               activation=hidden_filter,
                               kernel_regularizer=regularizers.l2(kernel_lam), 
                               bias_regularizer=regularizers.l2(bias_lam)))
        model.add(BatchNormalization())
    model.add(layers.Dropout(0.8))
    model.add(layers.Dense(n_outputs, 
                           kernel_initializer=last_layer_init, 
                           activation=last_layer_filter,
                           kernel_regularizer=regularizers.l2(kernel_lam),
                           bias_regularizer=regularizers.l2(bias_lam)))
    return model
        
    
def custom_loss(fp_penalty_coef, fn_penalty_coef):
    # custom loss function that penalize false positive and negative differently
    def loss(y_true, y_pred):
        res = y_pred - y_true
        res = tf.where(res > 0, res * fp_penalty_coef, res * fn_penalty_coef)
        return K.mean(K.square(res))
    return loss

def fp_mae(y_true, y_pred):
    # custom metric that replace false negative with zero and return the mean of new vector
    res = y_pred - y_true
    res = tf.nn.relu(res)
#     res = tf.where(res <= 0, 0, res)
    return K.mean(res)

def cus_mae(y_true, y_pred):
    def log10(x):
        numerator = K.log(x)
        denominator = K.log(K.constant(10, dtype=numerator.dtype))
        return numerator / denominator
    p_true = K.sum(K.pow(10.0, y_true/10))
    p_pred = K.sum(K.pow(10.0, y_pred/10))
    return K.mean(K.abs(10 * log10(p_true) - 10 * log10(p_pred)))


In [ ]:
# CNN + NN(multi-SUs): support batching
TEST, CONSERVE = True, False
mini_batch = 64 if max(max_x, max_y) == 1000 else 16
epochs = 35 if max(max_x, max_y) == 1000 else 300
MAX_QUEUE_SIZE, WORKERS = 28, 4
fp_penalty_coef, fn_penalty_coef = 1, 1
hyper_metric, mode = "val_cus_mae", 'min'  # the metric that hyper parameters are tuned with
lambda_vec = [0, 0.01, 0.1, 1, 10] 
prev_sample = 0
average_diff_power, fp_mean_power = [],[] #[7.177, 8.088, 8.183], [3.438, 3.506, 2.662]
best_lambda = []
average_diff_power_conserve, fp_mean_power_conserve = [], []
total_power, max_min_ratio = [], []
number_samples = [128, 256, 512, 1024, 2048, 4096]

for num_sample_idx, number_sample in enumerate(number_samples):
    MODEL_PATH = f"{dataset_path}/{number_sample}/models"
    X = np.loadtxt(f"{dataset_path}/{number_sample}/X.txt", delimiter=",")
    y = np.loadtxt(f"{dataset_path}/{number_sample}/y.txt", delimiter=",")
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)
    MODEL_PATH += "/best_model_lambda_"
    if True:
        nns = [nn_model(max_sus_num * 3, max_sus_num, kernel_lam=lamb, bias_lam=0,
                        num_hidden_layers=4, num_neurons=200)
               for lamb in lambda_vec]
        for nn in nns:
#             cnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','mae', fp_mean])
            # =optimizers.SGD(lr=0.1, momentum=0.9, decay=0.1/epochs, nesterov=False)
            nn.compile(loss="mse", 
                        optimizer=optimizers.Adam(), 
                        metrics=['mse', 'mae', cus_mae])
        checkpointers = [ModelCheckpoint(filepath=MODEL_PATH + str(lamb)+ '.h5',
                                         verbose=0, save_best_only=True, 
                                         monitor=hyper_metric,
                                         mode=mode)
                         for lamb in lambda_vec]
    else:
        cnns = []
        cnns = [models.load_model(MODEL_PATH + str(lamb) + '.h5', 
                                  custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
                                                  'fp_mae': fp_mae }) 
                for lamb in lambda_vec]
    number_start = time.time()
    val_size = math.ceil(number_sample * validation_size)
  
    print('number_samples:', number_sample, ", New samples:", number_sample - prev_sample)
    print("Validation size:", val_size, ", starts:", number_sample, ", ends:", number_sample + val_size - 1)
    
    for lamb_idx, lamb in enumerate(lambda_vec):
#     for lamb_idx, lamb in enumerate(lambda_vec[:len(lambda_vec) - num_sample_idx//2]):
#         if num_sample_idx == 3 and lamb_idx < 4:
#             continue
        lambda_start = time.time()
        nns[lamb_idx].fit(x=X[:number_sample+val_size,:],
                           y=y[:number_sample+val_size,:],
                           epochs=epochs, verbose=0,
                           validation_split=validation_size/(1 + validation_size), 
                           shuffle=True, callbacks=[checkpointers[lamb_idx]], 
                           workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, 
                           use_multiprocessing=False)
        
        print("\nLambda:", lamb, ", Time:", str(datetime.timedelta(seconds=int(time.time() - lambda_start))))
        print("Train Error(all epochs):", min(nns[lamb_idx].history.history['mae']), '\n', 
              [round(val, 3) for val in nns[lamb_idx].history.history['mae']])
#         print("Train FP Error(all epochs):", min(cnns[lamb_idx].history.history['fp_mae']), '\n',
#               [round(val,3) for val in cnns[lamb_idx].history.history['fp_mae']])
        print("Val Error(all epochs):", min(nns[lamb_idx].history.history['val_mae']), '\n', 
              [round(val,3) for val in nns[lamb_idx].history.history['val_mae']])
        print("Val custom mae Error(all epochs):", min(nns[lamb_idx].history.history['val_cus_mae']), '\n',
              [round(val,3) for val in nns[lamb_idx].history.history['val_cus_mae']])
    
    
    models_min_mae = [min(nns[lam_idx].history.history[hyper_metric]) for
                      lam_idx,_ in enumerate(lambda_vec)]
    best_lamb_idx = models_min_mae.index(min(models_min_mae))
    best_lambda.append(lambda_vec[best_lamb_idx])
    print("\nTrainig set size:", number_sample, ", Time:", str(datetime.timedelta(seconds=int(time.time() - 
                                                                                              number_start))),
          ", best_lambda:", lambda_vec[best_lamb_idx], ", min_" , ("fp_" if hyper_metric == "val_fp_mae" else ""),
          "error:", round(min(models_min_mae), 3))
    all_cnns.append(nns)
    del nns
    
    if TEST:
        # evaluating test images
        best_model = None
        best_model = models.load_model(MODEL_PATH + str(lambda_vec[best_lamb_idx]) + '.h5', 
                                       custom_objects={'mae': 'mae', 'mse': 'mse', 'cus_mae': cus_mae})
#                                        custom_objects={ 'loss': custom_loss(fp_penalty_coef, fn_penalty_coef), 
#                                                        'fp_mae': fp_mae,
#                                                       'mae':'mae', 'mse':'mse'})

        print("Test starts: ", number_sample + val_size, ", ends: ", data_reg.shape[0] - 1)
        time.sleep(1)
#         test_res = best_model.evaluate(test_generator, verbose=1, 
#                                        workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, use_multiprocessing=False)
        y_hat = best_model.predict(X[number_sample + val_size:, :], verbose=1, 
                                       workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, use_multiprocessing=False)
        total_power.append(np.mean(10 * np.log10(np.sum(10**(y_hat/10), axis=1))))
        max_min_ratio.append(np.mean(np.max(y_hat, axis=1) - np.min(y_hat, axis=1)))
        average_diff_power.append(np.mean(np.abs(
            10 * np.log10(np.sum(10 ** (y_hat/10), axis=1)) -
            10 * np.log10(np.sum(10 ** (y[number_sample + val_size:]/10), axis=1)))))
        
#         test_mae_idx, test_fp_mae_idx = [best_model.metrics_names.index(mtrc) 
#                                          for mtrc in ['mae','fp_mae']]
#         test_mae, test_fp_mae = test_res[test_mae_idx], test_res[test_fp_mae_idx]
#         average_diff_power.append(round(test_mae, 3))
#         fp_mean_power.append(round(test_fp_mae, 3))
        print('total_power: ', total_power[-1], ', average_difference: ', average_diff_power[-1],
              'max_min_ratio:', max_min_ratio[-1])
        
        
        print("\n\n")

        
#         var_f = open('/'.join(image_dir.split('/')[:-1]) +  '/models/' + model_name + "/"
#                      + intensity_degradation + '_' + str(slope) + '_' + 
#                      dtime + ".dat", "wb") # file for saving results
#         pickle.dump([average_diff_power, fp_mean_power, number_samples, best_lambda, 
#                      dataset_name, max_dataset_name, average_diff_power_conserve, fp_mean_power_conserve,
#                      checkpointers],
#                     file=var_f)
#         var_f.close()
        del best_model
#     prev_sample = number_sample

In [ ]:
print("average_diff_power:", [round(x, 3) for x in average_diff_power])
print("total_power:", [round(x, 3) for x in total_power])
print("max_min_ratio:", [round(x, 3) for x in max_min_ratio])
print(best_lambda)

In [ ]:
# Multi-SU using single deep-alloc and Bi-LSTM
# DataBatchGenerator

class LSTMBatchGenerator(Sequence):
    def __init__(self, dataset, batch_size: int, start_idx: int, float_memory_used,
                 image_dir: str, cnn_feature_model, num_sus, cnn_output_size):
        self.dataset = dataset
        self.batch_size, self.start_idx = batch_size, start_idx
#         self.number_image_channels, self.max_x, self.max_y = number_image_channels, max_x, max_y
        self.float_memory_used = float_memory_used
        self.image_dir = image_dir
        self.cnn_feature_model = cnn_feature_model
        self.num_sus = num_sus
        self.cnn_output_size = cnn_output_size
    
    def __len__(self):
        return np.ceil(self.dataset.shape[0] / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        size = min(self.dataset.shape[0] - idx * self.batch_size, self.batch_size)
        batch_x = np.empty((size, self.num_sus, self.cnn_output_size),
                           dtype=self.float_memory_used)
        batch_y = np.copy(self.dataset[idx * self.batch_size: idx * self.batch_size + size, 3::3])
        batch_x2 = np.copy(batch_y)
        batch_x2[:, 1:] = batch_x2[:, :-1]
        batch_x2[:,0] = 0
        for i in range(size):
            for su_idx in range(self.num_sus):
                imm = read_image((self.start_idx + idx * self.batch_size + i) * self.num_sus + su_idx,
                                 self.image_dir)
                batch_x[i][su_idx] = self.cnn_feature_model.predict(imm)[0]
        return ([batch_x, batch_x2.reshape(size, self.num_sus, 1)], batch_y.reshape(size, self.num_sus, 1))

def create_bilstm_model(units, num_sus, cnn_output_size):
    model = Sequential()
    model.add(layers.Bidirectional(layers.LSTM(units, return_sequences=True), 
                                   input_shape=(num_sus, cnn_output_size)))
    model.add(layers.Bidirectional(layers.LSTM(units)))
    model.add(layers.Dense(1, activation='linear'))
    return model

def create_rnn_encoder_decoder(units, num_sus, cnn_output_size):
    # define bi-lstm encoder
    encoder_inputs = Input(shape=(None, cnn_output_size))
    encoder = layers.Bidirectional(layers.LSTM(units, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    state_h = layers.Concatenate()([forward_h, backward_h])
    state_c = layers.Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    
    # define training decoder
    decoder_inputs = Input(shape=(None, 1))
    decoder_lstm = layers.LSTM(units * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = layers.Dense(1, activation='linear')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    return model

def create_inference_model(model, units):
    #inference encoder
    encoder_inputs = model.input[0] #input_1
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = model.layers[1].output # lstm_1
    state_h = layers.Concatenate()([forward_h, backward_h])
    state_c = layers.Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    encoder_model = Model(encoder_inputs, encoder_states)
    
    #inference decoder
    decoder_inputs = model.input[1] #input_2
    decoder_state_input_h = Input(shape=(units * 2,),name='input_3')
    decoder_state_input_c = Input(shape=(units * 2,),name='input_4')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_lstm = model.layers[5]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[6]
    decoder_outputs=decoder_dense(decoder_outputs)

    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    return encoder_model, decoder_model

def predict_su_adjusted_power(inf_enc, inf_dec, num_sus, data):
    # encode
    state = inf_enc.predict(data)
    # start of sequence input
    target_seq = np.array([0.0]).reshape(1, 1, 1)
    # collect predictions
    output = list()
    for _ in range(num_sus):
        # predict next char
        yhat, h, c = inf_dec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)

model_name = "log_vgg16_max_su_total"
max_su_image_dir = '/'.join(image_dir.split('/')[:-1]) + "/" + model_name + "/images"
number_samples = [2048, 4096]
hyper_metric, mode = "val_cus_mae", 'min'
MAX_QUEUE_SIZE, WORKERS = 28, 4
lstm_units = 128
mini_batch, epochs = 32, 100
fp_penalty_coef, fn_penalty_coef = 1, 1
dataset_path = '/'.join(image_dir.split('/')[:-1]) + "/" + model_name
fp_penalty_coef, fn_penalty_coef = 1, 1
# total_power, avg_diff_power, max_min_ratio = [], [], []
cnn_output_size = 512
dataset_path = '/'.join(image_dir.split('/')[:-1]) + "/" + model_name


model_path = "/home/shahrokh/projects/MLSpectrumAllocation/ML/data/pictures_299_299_transfer/" + \
             "splat/pu_circle_su_circle_60/raw_power_min_max_norm/color/log_pu5_su5/" + \
             "pus_1_sus_3_channels/models/log_vgg16/"
MODEL_PATH = f"{dataset_path}/bilstm/{lstm_units}_units"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
for number_sample in number_samples:
    cnn_model = models.load_model(f"{model_path}/{number_sample}/best_model_lambda_0.h5",
                                  custom_objects={ 
                                      'loss': custom_loss(fp_penalty_coef, fn_penalty_coef),
                                      'fp_mae': fp_mae,
                                      'mae':'mae', 'mse':'mse'})
    feature_output = Model(inputs=cnn_model.layers[0].input, outputs=cnn_model.layers[2].output)
    time_start = time.time()
    train_generator = LSTMBatchGenerator(
        dataset=data_max_su_tot[:number_sample],
        batch_size=mini_batch,
        start_idx=prev_sample, 
        float_memory_used=float_memory_used,
        cnn_feature_model=feature_output,
        num_sus=max_sus_num,
        cnn_output_size=cnn_output_size,
        image_dir=max_su_image_dir)
    

    val_size = math.ceil(number_sample * validation_size)
#     val_size = data_reg.shape[0] - number_sample
    val_generator = LSTMBatchGenerator(
        dataset=data_max_su_tot[number_sample:number_sample+val_size],
        batch_size=mini_batch,
        start_idx=number_sample, 
        float_memory_used=float_memory_used,
        cnn_feature_model=feature_output,
        num_sus=max_sus_num,
        cnn_output_size=cnn_output_size,
        image_dir=max_su_image_dir)
  
    print('number_samples:', number_sample, ", New samples:", number_sample - prev_sample)
    print("Validation size:", val_size, ", starts:", number_sample, ", ends:", number_sample + val_size - 1)
#     bilstm_model = create_bilstm_model(128, max_sus_num, cnn_output_size)
    train_model = create_rnn_encoder_decoder(units=lstm_units, num_sus=max_sus_num,
                                             cnn_output_size=cnn_output_size)
    
    checkpointer = ModelCheckpoint(filepath=MODEL_PATH + f"/best_model_{number_sample}.h5",
                                   verbose=1, save_best_only=True, 
                                   monitor=hyper_metric,
                                   mode=mode)
    train_model.compile(loss="mse", 
                        optimizer=optimizers.Adam(), 
                        metrics=['mse', 'mae', cus_mae])
    train_model.fit(train_generator, epochs=epochs, verbose=2,
                     validation_data=val_generator, 
                     shuffle=True, callbacks=[checkpointer], 
                     workers=WORKERS, max_queue_size=MAX_QUEUE_SIZE, 
                     use_multiprocessing=False)
    # re-loading best model
    best_lstm_mode = models.load_model(MODEL_PATH + f"/best_model_{number_sample}.h5",
                                       custom_objects={'mse':'mse', 'mae': 'mae',
                                                       'cus_mae': cus_mae})
    inf_enc, inf_dec = create_inference_model(model=best_lstm_mode, units=lstm_units)
    
    # predicting test-sample one-by-one
    test_generator = LSTMBatchGenerator(
        dataset=data_max_su_tot[number_sample+val_size:],
        batch_size=1,
        start_idx=number_sample+val_size, 
        float_memory_used=float_memory_used,
        cnn_feature_model=feature_output,
        num_sus=max_sus_num,
        cnn_output_size=cnn_output_size,
        image_dir=max_su_image_dir)
    tot_power_tmp, max_min_ratio_tmp, avg_diff_power_tmp = [], [], []
    
    for test_idx in range(number_sample+val_size, data_max_su_tot.shape[0]):
        X, y = test_generator.__getitem__(test_idx - (number_sample+val_size))
        y_hat = predict_su_adjusted_power(inf_enc, inf_dec, max_sus_num, X[0])
        
        y_hat_tot_power = 10 * np.log10((10 ** (y_hat/10)).sum())
        y_tot_power = 10 * np.log10((10 ** (y/10)).sum())
        
        tot_power_tmp.append(y_hat_tot_power)
        max_min_ratio_tmp.append(y_hat.max() - y_hat.min())
        avg_diff_power_tmp.append(abs(y_hat_tot_power - y_tot_power))
    
    total_power.append(sum(tot_power_tmp) / len(tot_power_tmp))
    avg_diff_power.append(sum(avg_diff_power_tmp) / len(avg_diff_power_tmp))
    max_min_ratio.append(sum(max_min_ratio_tmp) / len(max_min_ratio_tmp))
    
    print(f"number_sample: {number_sample}, total_power: {total_power[-1]}, avg_diff_power: {avg_diff_power[-1]}"
          f"max_min_ratio: {max_min_ratio[-1]}")
        

In [ ]:
def bilstm_inference(inf_enc, inf_dec, test_generator, start_idx, end_idx, results):
    inf_enc.summary()
    tot_power_tmp, max_min_ratio_tmp, avg_diff_tmp = [], [], []
    for test_idx in tqdm.tqdm(range(start_idx, end_idx)):
        X, y = test_generator.__getitem__(test_idx)
        y_hat = predict_su_adjusted_power(inf_enc, inf_dec, max_sus_num, X[0])
        
        y_hat_tot_power = 10 * np.log10((10 ** (y_hat/10)).sum())
        y_tot_power = 10 * np.log10((10 ** (y/10)).sum())
        
        tot_power_tmp.append(y_hat_tot_power)
        max_min_ratio_tmp.append(y_hat.max() - y_hat.min())
        avg_diff_power_tmp.append(abs(y_hat_tot_power - y_tot_power))
    results.append((tot_power_tmp, max_min_ratio_tmp, avg_diff_tmp))
jobs = []
test_size = data_max_su_tot.shape[0] - 25905 # number_sample - val_size
proc_sizes = [test_size//number_of_proccessors] * (number_of_proccessors)
proc_sizes[-1] += test_size % number_of_proccessors
proc_idx = [(sum(proc_sizes[:i])+ 25905 - (number_sample+val_size), 
             sum(proc_sizes[:i+1]) - (number_sample+val_size)+ 25905 -1) for i in range(number_of_proccessors)]
results = []

for i in range(number_of_proccessors):
    p = Process(target=bilstm_inference, 
                args=(inf_enc, inf_dec, test_generator, proc_idx[i][0], proc_idx[i][1], results))
    jobs.append(p)
    p.start()
for i in range(number_of_proccessors):
    jobs[i].join()

for i in range(number_of_proccessors):
    jobs[i].terminate()
    jobs[i].close()
del jobs

In [ ]:
model = Model(inputs=model.layers[0].input, outputs=model.layers[2].output)

In [ ]:
def multi_calc(data_arr):
    min_data, tot_power = [], []
    for data in data_arr:
        min_arr = [(sorted(val, key=lambda x:x[1])[-1][1] - sorted(val, key=lambda x:x[1])[0][1]) for val in data]
        min_data.append(sum(min_arr)/len(min_arr))
        sum_arr = [dec_to_db(sum([db_to_dec(val[1]) for val in fair_res_sng]))for fair_res_sng in data]
        tot_power.append(sum(sum_arr)/len(sum_arr))
    return [round(min_, 2) for min_ in min_data], [round(tot,2) for tot in tot_power]

In [ ]:
#### MULTIPLE - SUS
def db_to_dec(db: float):
    return 10 ** (db/10)
def dec_to_db(dec: float):
    return -float('inf') if dec <= 0 else 10 * math.log10(dec)

def greedy_sus(pus, requesting_sus, model, pl_info, number_channel = 1):
    # pus: (x, y, p), requesting_sus: (x, y)
    def best_su_candidate(pus, active_sus, requesting_sus):
        if len(requesting_sus) == 1:
            return requesting_sus[0][0]
        # active_sus: (x, y, allocated_power), requesting_sus: (req_id, x, y)
        power_map_cell_size, area_size = 50, 1000
        cell_weight, neighbour_weight = 0.5, 0.5
        power_map = [[0] * int(area_size // power_map_cell_size) for _ in range(int(area_size // power_map_cell_size))]
        
        for pu_num in range(int(len(pus)//3)):
            x, y, dec_p = pus[pu_num*3], pus[pu_num * 3 + 1], db_to_dec(pus[pu_num*3 + 2])
            power_map[int(x * cell_size // power_map_cell_size)][int(y * cell_size // power_map_cell_size)] += dec_p
        for su in active_sus:
            x, y, dec_p = su[0], su[1], db_to_dec(su[2])
            power_map[int(x * cell_size // power_map_cell_size)][int(y * cell_size // power_map_cell_size)] += dec_p
        
        power_score = [[0] * len(power_map[0]) for _ in range(len(power_map))]
        # updating power score
        for i in range(len(power_map)):
            for j in range(len(power_map[0])):
                power_score[i][j] = cell_weight * power_map[i][j] + neighbour_weight * sum(
                [power_map[ii][jj] for ii in range(max(0, i - 1), min(i + 1, len(power_map))) 
                 for jj in range(max(0, j - 1), min(j + 1, len(power_map[0]))) if not (i == ii and j == jj)])
        # find su with lowest weight
        best_su_req, min_score = -1, float('inf')
        for req_id, x, y in requesting_sus:
            if power_score[int(x * cell_size // power_map_cell_size)][int(x * cell_size // power_map_cell_size)] < min_score:
                min_score = power_score[int(x * cell_size // power_map_cell_size)][int(x * cell_size // power_map_cell_size)]
                best_su_req = req_id
        return best_su_req         
    active_sus = [{} for _ in range(number_channel)] # (request_id: power_allocated)
    assigned_req = set()
    for _ in range(len(requesting_sus)):
        max_allocated_power, best_channel, best_req_id = -float('inf'), -1, -1
        for ch in range(number_channel):
            nex_req_id = best_su_candidate(pus, [(*requesting_sus[i], active_sus[ch][i]) for i in active_sus[ch]],
                                          [(idd, *requesting_sus[idd]) for idd in range(len(requesting_sus))
                                           if idd not in assigned_req])
            su_lst = []
            for i in active_sus[ch]:
                su_lst += [*requesting_sus[i]]
                su_lst.append(active_sus[ch][i])
            requesting_data = [len(pus)//3] + pus + [len(active_sus[ch]) + 1] + su_lst + requesting_sus[nex_req_id]
            requesting_image = np.expand_dims(create_image(requesting_data, slope=slope, style=style, 
                                                           noise_floor=noise_floor, pu_shape=pu_shape, su_shape=su_shape,
                                                           su_param=su_param, sensors_num=(sensors_num if sensors else 0), 
                                                           intensity_degradation=intensity_degradation, 
                                                           max_pu_power=0.0 if not sensors else -30,
                                                           max_su_power=0.0), 0)
            predicted_power = model.predict(requesting_image)
            if predicted_power[0][0] > max_allocated_power:
                max_allocated_power = predicted_power[0][0]
                best_channel, best_req_id = ch, nex_req_id
        active_sus[best_channel][best_req_id] = max_allocated_power
        assigned_req.add(best_req_id)
    res = []
    for ch in range(number_channel):
        for req_id in active_sus[ch]:
            res.append((req_id, active_sus[ch][req_id], ch))
    return res

def fair_sus(pus, requesting_sus, model, pl_info, number_channel = 1):
    # pus: (x, y, p), requesting_sus: (x, y)
    def best_su_candidate(pus, active_sus, requesting_sus):
        if len(requesting_sus) == 1:
            return requesting_sus[0][0]
        # active_sus: (x, y, allocated_power), requesting_sus: (req_id, x, y)
        power_map_cell_size, area_size = 50, 1000
        cell_weight, neighbour_weight = 0.5, 0.5
        power_map = [[0] * int(area_size // power_map_cell_size) for _ in range(int(area_size // power_map_cell_size))]
        
        for pu_num in range(int(len(pus)//3)):
            x, y, dec_p = pus[pu_num*3], pus[pu_num * 3 + 1], db_to_dec(pus[pu_num*3 + 2])
            power_map[int(x * cell_size // power_map_cell_size)][int(y * cell_size // power_map_cell_size)] += dec_p
        for su in active_sus:
            x, y, dec_p = su[0], su[1], db_to_dec(su[2])
            power_map[int(x * cell_size // power_map_cell_size)][int(y * cell_size // power_map_cell_size)] += dec_p
        
        power_score = [[0] * len(power_map[0]) for _ in range(len(power_map))]
        # updating power score
        for i in range(len(power_map)):
            for j in range(len(power_map[0])):
                power_score[i][j] = cell_weight * power_map[i][j] + neighbour_weight * sum(
                [power_map[ii][jj] for ii in range(max(0, i - 1), min(i + 1, len(power_map))) 
                 for jj in range(max(0, j - 1), min(j + 1, len(power_map[0]))) if not (i == ii and j == jj)])
        # find su with lowest weight
        best_su_req, min_score = -1, float('inf')
        for req_id, x, y in requesting_sus:
            if power_score[int(x * cell_size // power_map_cell_size)][int(x * cell_size // power_map_cell_size)] < min_score:
                min_score = power_score[int(x * cell_size // power_map_cell_size)][int(x * cell_size // power_map_cell_size)]
                best_su_req = req_id
        return best_su_req         
    active_sus = [{} for _ in range(number_channel)] # (request_id: power_allocated)
    assigned_req = set()
    for _ in range(len(requesting_sus)):
        max_allocated_power, best_channel, best_req_id = -float('inf'), -1, -1
        for ch in range(number_channel):
            nex_req_id = best_su_candidate(pus, [(*requesting_sus[i], active_sus[ch][i]) for i in active_sus[ch]],
                                          [(idd, *requesting_sus[idd]) for idd in range(len(requesting_sus))
                                           if idd not in assigned_req])
            su_lst = []
            for i in active_sus[ch]:
                su_lst += [*requesting_sus[i]]
                su_lst.append(active_sus[ch][i])
            requesting_data = [len(pus)//3] + pus + [len(active_sus[ch]) + 1] + su_lst + requesting_sus[nex_req_id]
            requesting_image = np.expand_dims(create_image(requesting_data, slope=slope, style=style, 
                                                           noise_floor=noise_floor, pu_shape=pu_shape, su_shape=su_shape,
                                                           su_param=su_param, sensors_num=(sensors_num if sensors else 0), 
                                                           intensity_degradation=intensity_degradation, 
                                                           max_pu_power=0.0 if not sensors else -30,
                                                           max_su_power=0.0), 0)
            predicted_power = model.predict(requesting_image)
            if predicted_power[0][0] > max_allocated_power:
                max_allocated_power = predicted_power[0][0]
                best_channel, best_req_id = ch, nex_req_id
        if max_allocated_power < -20:
            # it's less than threshold. sort active su w.r.t to best_req_id and try to decrease their power
            def dist(p1, p2):
                return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5
            dist_info = []
            for ch in range(number_channel):
                for i in active_sus[ch]:
                    dist_info.append((ch, i, dist(requesting_sus[i], requesting_sus[best_req_id])))
            dist_info.sort(key=lambda x:x[2])
            SATISFIED = False
            for i in range(len(dist_info)):
                candid_ch, candid_su_id  = dist_info[i][0], dist_info[i][1]
                candid_old_pow = active_sus[candid_ch][candid_su_id]
                if  candid_old_pow > -10:
                    candid_new_pow = candid_old_pow
                    while candid_new_pow - 5 > -20.0:
                        candid_new_pow -= 5
                        # try this new power
                        su_lst = []
                        for ii in active_sus[candid_ch]:
                            su_lst += [*requesting_sus[ii]]
                            if ii == candid_su_id:
                                su_lst.append(candid_new_pow)
                            else:
                                su_lst.append(active_sus[candid_ch][ii])
                        requesting_data = [len(pus)//3] + pus + [len(active_sus[candid_ch]) + 1] + su_lst + requesting_sus[best_req_id]
                        requesting_image = np.expand_dims(create_image(requesting_data, slope=slope, style=style, 
                                                                       noise_floor=noise_floor, pu_shape=pu_shape, su_shape=su_shape,
                                                                       su_param=su_param, sensors_num=(sensors_num if sensors else 0), 
                                                                       intensity_degradation=intensity_degradation, 
                                                                       max_pu_power=0.0 if not sensors else -30,
                                                                       max_su_power=0.0), 0)
                        predicted_power = model.predict(requesting_image)[0][0]
                        if predicted_power > -20.0:
                            SATISFIED = True
                            break
                    if SATISFIED:
                        break
            if SATISFIED:
                best_channel, max_allocated_power = candid_ch, predicted_power
                active_sus[candid_ch][candid_su_id] = candid_new_pow  #update the candid su
                    
        active_sus[best_channel][best_req_id] = max_allocated_power
        assigned_req.add(best_req_id)
    res = []
    for ch in range(number_channel):
        for req_id in active_sus[ch]:
            res.append((req_id, active_sus[ch][req_id], ch))
    return res

def random_sus(pus, requesting_sus, model, pl_info, number_channel = 1):
    active_sus = [{}  for _ in range(number_channel)] # (request_id: power_allocated)
    assigned_req = set()
    for nex_req_id in range(len(requesting_sus)):
        max_allocated_power, best_channel, best_req_id = -float('inf'), -1, -1
        for ch in range(number_channel):
            su_lst = []
            for i in active_sus[ch]:
                su_lst += [*requesting_sus[i]]
                su_lst.append(active_sus[ch][i])
            requesting_data = [len(pus)//3] + pus + [len(active_sus[ch]) + 1] + su_lst + requesting_sus[nex_req_id]
            requesting_image = np.expand_dims(create_image(requesting_data, slope=slope, style=style, 
                                                           noise_floor=noise_floor, pu_shape=pu_shape, su_shape=su_shape,
                                                           su_param=su_param, sensors_num=(sensors_num if sensors else 0), 
                                                           intensity_degradation=intensity_degradation, 
                                                           max_pu_power=0.0 if not sensors else -30,
                                                           max_su_power=0.0), 0)
            predicted_power = model.predict(requesting_image)
            if predicted_power[0][0] > max_allocated_power:
                max_allocated_power = predicted_power[0][0]
                best_channel, best_req_id = ch, nex_req_id
        active_sus[best_channel][best_req_id] = max_allocated_power
        assigned_req.add(best_req_id)
    res = []
    for ch in range(number_channel):
        for req_id in active_sus[ch]:
            res.append((req_id, active_sus[ch][req_id], ch))
    return res
    
def multiple_sus(data_reg, train_set_size, pl_info, model_path, number_channel):
    random_res, greedy_res = [[] for _ in range(len(train_set_size))], [[] for _ in range(len(train_set_size))]
    fair_res = [[] for _ in range(len(train_set_size))]
    for ind, train_size in enumerate(train_set_size):
        print(f"Train size: {train_size}")
        model = models.load_model(f"{model_path}/{train_size}/best_model_lambda_0.1.h5",
                                 custom_objects={ 'loss': custom_loss(1, 1), 
                                               'fp_mae': fp_mae,
                                               'mae':'mae', 'mse':'mse'})
        for i in tqdm.tqdm(range(len(data_reg))):
            pu_num = int(data_reg[i][0])
            pus = np.ndarray.tolist(data_reg[i][1:1 + pu_num * 3])
            su_num = int(data_reg[i][1 + pu_num * 3])
            if su_num < 4:
                continue
            sus = []
            for su_ind in range(su_num):
                sus.append(np.ndarray.tolist(data_reg[i][2 + pu_num * 3 + su_ind * 3:4 + pu_num * 3 + su_ind * 3]))

            fair_res[ind].append(fair_sus(pus, sus, model, pl_info, number_channel))
            random_res[ind].append(random_sus(pus, sus, model, pl_info, number_channel))
            greedy_res[ind].append(greedy_sus(pus, sus, model, pl_info, number_channel))
            var_f = open('/'.join(image_dir.split('/')[:-1]) + "/multi_res_num_channel"
                     + str(number_channel) + "_" + intensity_degradation + '_' + str(slope) + '_' + 
                     dtime + ".dat", "wb") # file for saving results
            pickle.dump([random_res, greedy_res, fair_res],  file=var_f)
            var_f.close()
            
#     return random_sum_power, random_max_min_ratio, greedy_sum_power, greedy_max_min_ratio
    return random_res, greedy_res, fair_res

In [ ]:
rand_res, greedy_res, fair_res = multiple_sus(data_reg[:600], number_samples, None,
                                              "ML/data/pictures_299_299_transfer/splat/pu_circle_su_circle_60/raw_power_min_max_norm/color/log_5/pus_5_sus_3_channels/models/log_vgg16",
                                              4)